In [11]:
import pandas as pd
import numpy as np
from pprint import pprint
import pickle
import glob, os
import nltk
from nltk.util import ngrams
from nltk import bigrams, trigrams, word_tokenize
from collections import Counter, defaultdict
import random
os.chdir("../quoteAIsData")

Using 4 gram N-Gram model

In [13]:
names = ["DAVID", "DIEGO", "ERIC", "GWYN", "JETT", "MARS", "MILES", "MILO", "PARSA"]
def safe_insert_list(dictionary, key, value):
    if key in dictionary:
        dictionary[key].append(value)
    else:
        dictionary[key] = list()
        dictionary[key].append(value)

def safe_insert_dict(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = {}
    if value not in dictionary[key]:
        dictionary[key][value] = 0

def line_counts(dictionary):
    for key in dictionary:
        print(key, len(dictionary[key]))
        
def getfileOwner(file):
    name = file.split('.')[0]
    for i in range(0, len(name)):
        if name[i].isdigit():
            name = name [:i]
            break
    return name.upper()


def print_quotegen(quote_gen):
    for k,v  in quote_gen.items():
        print(k)
        print('\t',[ (k2,v2) for k2,v2  in v.items()])
        
def load_quotegen(person):
    return pickle.load(open(person + '.pkl', 'rb') )


In [15]:
quote_dict = {}
for file in glob.glob("*.txt"):
    #not a base file
    if "_" in file:
        continue
    owner = getfileOwner(file)
    f = open(file, 'r', encoding="utf-8")
    lines = f.read()
    lines = lines.split("\n")
    for l in lines:
        if l == "":
            continue
        safe_insert_list(quote_dict, owner, l)
    f.close()
for k in quote_dict:
    print(k, len(quote_dict[k]))

DAVID 2446
DIEGO 3260
ERIC 32224
GWYN 21457
JETT 5554
MARS 23434
MILES 15062
MILO 34714
PARSA 9714


In [16]:
def make_quotegen(person='PARSA'):   
    quote_gen = {}

    for sentence in quote_dict[person]:
        token = nltk.word_tokenize(sentence)
        for w1, w2, w3, w4 in ngrams(token, 4, pad_left = True, pad_right=True):
            safe_insert_dict(quote_gen, (w1, w2, w3), w4)
            quote_gen[(w1, w2, w3)][w4] += 1
    
    for w1_w2_w3 in quote_gen:
        total_count = float(sum(quote_gen[w1_w2_w3].values()))
        for w4 in quote_gen[w1_w2_w3]:
            quote_gen[w1_w2_w3][w4] /= total_count

    return quote_gen

for person in quote_dict:
    q = make_quotegen(person)
    pickle.dump(q, open(person + '.pkl', 'wb') )
    print("Done with quotegen " + person)

Done with quotegen DAVID
Done with quotegen DIEGO
Done with quotegen ERIC
Done with quotegen GWYN
Done with quotegen JETT
Done with quotegen MARS
Done with quotegen MILES
Done with quotegen MILO
Done with quotegen PARSA


In [17]:
def generate_sentence(person):
    quote_gen = load_quotegen(person)
    text = [None,None,None]
    sentence_finished = False
    while not sentence_finished:
        r = random.random()
        accumulator = .0

        for word in quote_gen[tuple(text[-3:])].keys():
            accumulator += quote_gen[tuple(text[-3:])][word]

            if accumulator >= r:
                text.append(word)
                break

        if text[-3:] == [None,None,None]:
            sentence_finished = True

    return ' '.join([t for t in text if t])

In [18]:
for i in range(20):
    print(generate_sentence('PARSA'))

idk how the other guys
hepy burf
how about a great sense of pride in oneself
so we all good for 8pm
its pretty easy but now that he 's lvl 6 he wants to transcend his humanity through `` the purity of metal ''
WOOOO SOLO
2 cantrips a turn 8 rolls
I identify as a gamer first and an American second
i had to explain to someone that alcohol was a poison . he didnt know that and im like why the fuck does it wear down your liver then ? that part of your body that deals with poison ?
cant go back tho
it keeps running in the background after you close it and it has access to your mic and camera
He gets a lot of wwe
maybe the Miracle takes people who suffer to change their burden and punishes those who are evil with brutal death
grim is your connection alright ?
do you remember that one time i was a spider for a game called Mordhau ?
yes
NOW YOURE GOING TO JAIL
no
Enjoy 4 more years
I do n't want to continue the game because clinkz retard rushed mid and killed our ancient in like 4 seconds


In [7]:
for n in names:
    print(f"{n} quotes:")
    for i in range(20):
        print('\t', generate_sentence(n))

DAVID quotes:
	 It was good .
	 what can I say except you & # 39 ; s ancient rather than hooked beak .
	 Watch your tone with me , boy . You invert Mario to create Luigi -- the same items and abilities
	 Cue
	 The modeling ones tho
	 I want to find out that you may have invented it
	 What did you link this random stream ?
	 Ads in Twitch ? WutFace
	 I do n't exist any more
	 That looks unsafe .
	 That was 100 % science thinking .
	 My opponent has quite a name
	 It 's not
	 The Czech Republic won the Global Games finals is between Ukraine and the price , you just tilted by the American Flag and shed a tear on the chalk . The storm that wipes out the runes .
	 🅱️🅱️🅱️🅱️🅱️🅱️🅱️🅱️🅱️🅱️🅱️🅱️🅱️🅱️🅱️
	 Join
	 ( smile )
	 J U D Y S
	 Asuka lover
	 `` Galio will ruin you ''
DIEGO quotes:
	 milos kais
	 idk , i just wanted to talk about , sorry I did n't mean to hurt you .
	 ♫ Digbo leans like a harmoinca
	 Oh , hell yeah . I probably already got 20+ street passes
	 Edited previous message : pepehan

In [19]:
pickle.dump(quote_dict, open('quotes.pkl', 'wb'))